# Env Setup

In [1]:
# Env Setup
import numpy as np
import cv2 as cv
from PIL import Image
import os
import epd_utils as epd
import cnt_white_edge as cwe
from collections import Counter
import dither

import tqdm
from numba import jit

In [33]:
# Preparation
path_wkspace = 'Exp/star/'
path_src_home = path_wkspace + 'src/'
path_out_home = path_wkspace + 'out/'
# path_out_home = '\\\\192.168.1.9\\myshare\\IT8951\\pic\\exp\\star\\'
path_out_home = '\\\\192.168.137.114\\myshare\\IT8951\\pic\\exp\\opera\\'

src_type = '.jpg'
dst_type = '.bmp'
A = 'A'
B = 'B'
path_src_A = path_src_home + A + src_type
path_src_B = path_src_home + B + src_type
path_dst_A = path_out_home + A + dst_type
path_dst_B = path_out_home + B + dst_type

img_A = cv.imread(path_src_A, cv.IMREAD_GRAYSCALE)
img_B = cv.imread(path_src_B, cv.IMREAD_GRAYSCALE)

# img_A = epd.threshold(img_A, 238, 238, mode=3)
# img_B = epd.threshold(img_A, 238, 238, mode=3)

img_A = dither.floyd_steinberg(img_A / 255) * 255
img_B = dither.floyd_steinberg(img_B / 255) * 255
# img_A = cv.rotate(img_A, cv.ROTATE_90_CLOCKWISE)
# img_B = cv.rotate(img_B, cv.ROTATE_90_CLOCKWISE)
# # #
# epd.saveAsBlackWhiteImage(path_dst_A, img_A)
# epd.saveAsBlackWhiteImage(path_dst_B, img_B)

In [3]:
# floodfill return filled pts
def fill(data, start_coords, fill_value, border_value, orig_val_bound=(10, 10), connectivity=8):
    """
    Flood fill algorithm

    Parameters
    ----------
    data : (M, N) ndarray of uint8 type
        Image with flood to be filled. Modified inplace.
    start_coords : tuple
        Length-2 tuple of ints defining (row, col) start coordinates.
    fill_value : int
        Value the flooded area will take after the fill.
    border_value: int
        Value of the color to paint the borders of the filled area with.
    connectivity: 4 or 8
        Connectivity which we use for the flood fill algorithm (4-way or 8-way).

    Returns
    -------
    filled_data: ndarray
        The data with the filled area.
    borders: ndarray
        The borders of the filled area painted with border_value color.
    """
    assert connectivity in [4, 8]

    filled_data = data.copy()
    # print(filled_data.shape)
    xsize, ysize = filled_data.shape
    orig_value = filled_data[start_coords[0], start_coords[1]]

    stack = set(((start_coords[0], start_coords[1]),))
    # if fill_value == orig_value:
    #     raise ValueError("Filling region with same value already present is unsupported. Did you already fill this region?")

    border_points = []
    filled_pts = []
    lower_bound, upper_bound = orig_val_bound
    lower_bound = orig_value - lower_bound
    upper_bound = orig_value + upper_bound
    upper_bound = upper_bound if upper_bound < 256 else 255
    lower_bound = lower_bound if lower_bound > -1 else 0
    while stack:
        x, y = stack.pop()
        # cond_fill = (filled_data[x, y] == orig_value)
        cond_fill = (filled_data[x, y] < upper_bound) and (filled_data[x, y] > lower_bound)
        if cond_fill:
            filled_data[x, y] = fill_value
            filled_pts.append((y,x))
            if x > 0:
                stack.add((x - 1, y))
            if x < (xsize - 1):
                stack.add((x + 1, y))
            if y > 0:
                stack.add((x, y - 1))
            if y < (ysize - 1):
                stack.add((x, y + 1))

            if connectivity == 8:
                if x > 0 and y > 0:
                    stack.add((x - 1, y - 1))
                if x > 0 and y < (ysize - 1):
                    stack.add((x - 1, y + 1))
                if x < (xsize - 1) and y > 0:
                    stack.add((x + 1, y - 1))
                if x < (xsize - 1) and y < (ysize - 1):
                    stack.add((x + 1, y + 1))
        else:
            if filled_data[x, y] != fill_value:
                border_points.append([x, y])

    # Fill all image with white
    borders = filled_data.copy()
    borders.fill(255)

    # Paint borders
    for x, y in border_points:
        borders[x, y] = border_value

    return filled_data, borders, set(filled_pts)

In [4]:
# get windowed White Edge matrix
def getAvgWinForWhiteEdge(img, ghost_threshold=14, step=(6, 6)):
    mImg = np.copy(img)
    h, w = mImg.shape
    # print(h,w)
    ghost_pts = {}
    x_step, y_step = step
    img3 = epd.createEmptyGrayScaleImage((int(h / y_step), int(w / x_step)), gray=0)
    acc = np.array([0])
    # plt.figure().set_size_inches(60, 80)
    for y in range(0, h, y_step):
        for x in range(0, w, x_step):
            avg = cv.mean(mImg[y:y + y_step, x:x + x_step])
            if avg[0] < ghost_threshold:
                continue
            mImg[y:y + y_step, x:x + x_step] = avg[0]
            img3[int(y / y_step), int(x / x_step)] = avg[0]
            ghost_pts[(int(x / x_step),int(y / y_step))] = avg[0]
        pass
    return mImg, img3, ghost_pts

In [5]:
# Visualize points in image
def colorPoint(img,pts,color):
    for p in pts:
        x,y = p
        r,g,b = color
        img[y,x,0] = r
        img[y,x,1] = g
        img[y,x,2] = b

def colorPoint2(img,pts,color):
    for p in pts:
        y,x = p
        r,g,b = color
        img[y,x,0] = r
        img[y,x,1] = g
        img[y,x,2] = b

def vizPtsInArea(shape,pts_list,color_list=None):
    canvas = np.zeros(shape=shape)
    canvas = cv.cvtColor(canvas.astype('uint8'), cv.COLOR_GRAY2RGB)
    canvas[:, :, :] = 255
    if color_list is None:
        color_list = [(0x66, 0x66, 0x99), (0xFF, 0xCC, 0), (0x99, 0xcc, 0x33), (0x99, 0x33, 0x66),
                      (0, 0x99, 0x66),(0x22, 0xFF, 0x66) , (0x22, 0x77, 0x66),(0x22, 0, 0x66),
                      (0xdd, 0xcc, 0xaa), (0, 0xff, 0xff), (0, 0xff, 0), (0x33, 0xff, 0xf0),
                      (0, 0, 0xff), (0xff, 0xff, 0), (0xff, 0, 0), (0, 0x33, 0x66), (0xff, 0x33, 0x66), (0xff, 0x33, 0xFF)]
    for i,pts in enumerate(pts_list):
        colorPoint(canvas,pts,color_list[i%len(color_list)])
    epd.showCv2Img(canvas)

## Get Downscaled Image B

In [6]:
# downscale img B
img_B_fullscale, img_B_downscale = epd.getAvgWin(img_B)

## White Edge Ghost

### Get white edge points

In [7]:
# downscale White Egde Layer
we_cnt_lut, we_pos_lut, we_mat012 = cwe.getWhiteEdgeCnt2(img_A, img_B)
we_mat255 = we_mat012 * 120
we_mat_fullscale, we_mat_downscale, we_ds_pts = getAvgWinForWhiteEdge(we_mat255, ghost_threshold=14)

#### Visualize White Edge points

In [8]:
# vizPtsInArea(we_mat012.shape, [list(we_cnt_lut.keys())])

#### Floodfill for same colored area

In [9]:
import time
h, w = we_mat_downscale.shape
ibd = np.copy(img_B_downscale)

we_ds_pt_set = set(we_ds_pts)
orig_area_pt_lists = []
we_area_pt_lists = []
area_lists = []
cnt = 0
t0 = time.time()
while len(we_ds_pt_set) > 0:
    p = we_ds_pt_set.pop()
    x1,y = p
    _, _, fpts = fill(ibd, (y, x1), 255, 0, orig_val_bound=(15, 15))
    if len(fpts) > 1:
        we_area_pts = we_ds_pt_set & fpts
        if (len(we_area_pts)<5):
            continue
        orig_area_pt_lists.append(fpts)
        we_area_pt_lists.append(we_area_pts)
        area_lists.append((fpts,we_area_pts))
    we_ds_pt_set = we_ds_pt_set - set(fpts)
print(time.time()-t0)

0.46610212326049805


In [10]:
# vizPtsInArea(we_mat_downscale.shape, orig_area_pt_lists)

#### Make region whiter

In [11]:
def getWeGray(we_cnt):
    return 2*we_cnt-0.5
# print(stat_gray_levels)

# @jit(nopython=True)
pts = []
path_wkspace = 'Exp/star/'
path_src_home = path_wkspace + 'src/'
path_out_home = path_wkspace + 'out/'
path_out_home = '\\\\192.168.137.114\\myshare\\IT8951\\pic\\exp\\opera\\'

src_type = '.jpg'
dst_type = '.bmp'
A = 'A'
B = 'B'
path_src_A = path_src_home + A + src_type
path_src_B = path_src_home + B + src_type
path_dst_B1 = path_out_home + 'B1' + dst_type
img_B = cv.imread(path_src_B, cv.IMREAD_GRAYSCALE)
img_B1 = np.copy(img_B)
def areaSubtract(img, pts_list, minus = 20):
    newB = np.copy(img)
    h,w = newB.shape
    for y in tqdm.tqdm(range(h)):
        for x in range(w):
            pix = newB[y,x]
            if x == 1030 and y ==1060:
                pass
            if ((x,y) in pts_list):
                tmp = pix - minus
                newB[y,x] = tmp if tmp > 0 else 0
    return newB


# img_B = areaSubtract(img_B, we_cnt_lut, 50) # amost ghost not visible
# img_B1 = areaSubtract(img_B1, we_cnt_lut, 30) # a little darker when diffuse white edge to 'y'
img_B1 = areaSubtract(img_B1, we_cnt_lut, 16)

imgB_after_we_darker = np.copy(img_B1) # for fllowing process
img_B1 = dither.floyd_steinberg(img_B1 / 255) * 255
img_B1 = cv.rotate(img_B1, cv.ROTATE_90_CLOCKWISE)
epd.saveAsBlackWhiteImage(path_dst_B1, img_B1)

100%|██████████| 1872/1872 [00:06<00:00, 269.55it/s]


#### Step2

In [12]:
tmpListLenAbove10000 = []
for l in orig_area_pt_lists:
    if len(l) > 10000:
        tmpListLenAbove10000.append(l)
# vizPtsInArea(we_mat_downscale.shape, tmpListLenAbove10000[0:3])

In [13]:
def addAreaWithVal(area,val):
    h,w = area.shape
    for y in range(h):
        for x in range(w):
            pix = area[y,x]
            tmp = pix + val
            tmp = tmp if tmp > 0 else 0
            tmp = tmp if tmp < 255 else 255
            area[y,x] = tmp
def makeAreaWhiter(img, ds_area_pts, ds_we_pts, val):
    new_img =  np.copy(img)
    add_mat = np.zeros_like(img)
    add_mat = add_mat + val
    pts = []
    for p in tqdm.tqdm(ds_area_pts):
        if p in ds_we_pts:
            continue
        x,y = p
        pts.append(p)
        # new_img[y*6:(y+1)*6-1,x*6:(x+1)*6-1] = new_img[y:y+6,x:x+6] + val
        # new_img[y*6:(y+1)*6-1,x*6:(x+1)*6-1] = 255#np.zeros_like(new_img[y:y+6,x:x+6]) + val
        # addAreaWithVal(new_img[y:y+6,x:x+6], 255)
        new_img[y*6:(y+1)*6-1,x*6:(x+1)*6-1] = cv.add(new_img[y*6:(y+1)*6-1,x*6:(x+1)*6-1], val)
    # vizPtsInArea(we_mat_downscale_after_we_darker.shape, [pts])
    return new_img

In [14]:
vizPtsInArea(we_mat_downscale.shape, [tmpListLenAbove10000[1]])

In [15]:
part1_nodither = makeAreaWhiter(imgB_after_we_darker, tmpListLenAbove10000[1], we_ds_pts, 8)
# part1 = imgB_after_we_darker
# part1[13:13+200,]
# epd.showCv2Img(part1)
# print(np.)
# part1[100:300,100:300] = 255
part1 = dither.floyd_steinberg(part1_nodither / 255) * 255
# part1 = cv.rotate(part1, cv.ROTATE_90_CLOCKWISE)
epd.saveAsBlackWhiteImage(path_dst_B1, part1)
print('Done')

100%|██████████| 13992/13992 [00:00<00:00, 440181.07it/s]


Done


In [16]:
part0_nodither = makeAreaWhiter(part1_nodither, tmpListLenAbove10000[0], we_ds_pts, 5)
# part1 = imgB_after_we_darker
# part1[13:13+200,]
# epd.showCv2Img(part1)
# print(np.)
# part1[100:300,100:300] = 255
part0 = dither.floyd_steinberg(part0_nodither / 255) * 255
part0 = cv.rotate(part0, cv.ROTATE_90_CLOCKWISE)
epd.saveAsBlackWhiteImage(path_dst_B1, part0)
print('Done')

100%|██████████| 11649/11649 [00:00<00:00, 582325.60it/s]


Done


In [17]:
part2_nodither = makeAreaWhiter(part0_nodither, tmpListLenAbove10000[2], we_ds_pts, 5)
part2 = dither.floyd_steinberg(part2_nodither / 255) * 255
part2 = cv.rotate(part2, cv.ROTATE_90_CLOCKWISE)
epd.saveAsBlackWhiteImage(path_dst_B1, part2)
print('Done')

100%|██████████| 16160/16160 [00:00<00:00, 557112.29it/s]


Done


In [18]:
img_B_Kill_WE = part2_nodither

## White Minus Ghost

### Visualize White Minus Ghost

In [19]:

def getWhiteMinusCnt(img_A, img_B):
    A = np.copy(img_A).astype('int16') / 255
    B = np.copy(img_B).astype('int16') / 255
    D = B - A
    D = D + 1
    D = D / 2
    h, w = A.shape
    wm_pts = []
    for y in range(h):
        for x in range(w):
            if D[y, x] == 1:
                wm_pts.append((x, y))
    return wm_pts


wm_pts = getWhiteMinusCnt(img_A, img_B)
# vizPtsInArea(we_mat012.shape, [wm_pts])

### Make white gray

#### Downscale

In [20]:
def getAvgWinForWhiteMinus(wm_pts, threshold=18):
    h = int(1872/6)
    w = int(1404/6)
    wm_ds_mat = np.zeros((h,w))
    wm_ds_pts = []
    for p in wm_pts:
        x,y = p
        x1 = int(x/6)
        y1 = int(y/6)
        wm_ds_mat[y1,x1] = wm_ds_mat[y1,x1] + 1
    for y in range(h):
        for x in range(w):
            if wm_ds_mat[y,x] > threshold:
                wm_ds_pts.append((x,y))
    return wm_ds_pts,wm_ds_mat

In [21]:
wm_ds_pts,wm_ds_mat = getAvgWinForWhiteMinus(wm_pts, threshold=6)

In [22]:
# vizPtsInArea(we_mat_downscale.shape, [wm_ds_pts])

#### Find Same Color Areas

In [23]:
import time
h, w = we_mat_downscale.shape
ibds = np.copy(img_B_downscale)

wm_ds_pt_set = set(wm_ds_pts)
orig_area_pt_lists2 = []
we_area_pt_lists = []
area_lists = []
cnt = 0
t0 = time.time()
while len(wm_ds_pt_set) > 0:
    # print(len(wm_ds_pt_set))
    p = wm_ds_pt_set.pop()
    x1,y = p
    _, _, fpts = fill(ibds, (y, x1), 255, 0, orig_val_bound=(15, 15))
    if len(fpts) > 1:
        wm_area_pts = wm_ds_pt_set & fpts
        # if (len(wm_area_pts)<5):
        #     continue
        orig_area_pt_lists2.append(fpts)
        we_area_pt_lists.append(wm_area_pts)
        area_lists.append((fpts,wm_area_pts))
    wm_ds_pt_set = wm_ds_pt_set - set(fpts)
print(time.time()-t0)

0.9052581787109375


In [24]:
# vizPtsInArea(we_mat_downscale.shape, orig_area_pt_lists2)


In [25]:
# _, _, fpts = fill(ibds, (243, 214), 255, 0, orig_val_bound=(20, 20))
wpts = []
h,w = img_B_Kill_WE.shape
for y in range(h):
    for x in range(w):
        if img_B_Kill_WE[y,x] > 238:
            wpts.append((x,y))

WhitePts = wpts
# vizPtsInArea(we_mat_fullscale.shape, [fpts])



In [26]:
WhitePts = wpts

In [27]:
WhitePtsExceptWM = list(set(WhitePts) - set(wm_pts))
# vizPtsInArea(we_mat_fullscale.shape, [WhitePtsExceptWM])


#### Make White Minus Area Grayer

In [28]:
print(img_B_Kill_WE.shape)
print(img_B.shape)


(1872, 1404)
(1872, 1404)


In [29]:
def makeAreaGrayer(img, pts, val):
    new_img =  np.copy(img)
    for p in tqdm.tqdm(pts):
        x,y = p
        tmp = new_img[y,x] - val
        new_img[y,x] = tmp if tmp > 0 else 0
    return new_img

In [30]:
WM2GRAY = np.copy(img_B_Kill_WE)
# part1[100:300,100:300] = 255

# wm2gray = makeAreaGrayer(WM2GRAY, wm_pts, 10)
# wm2gray = makeAreaGrayer(wm2gray, WhitePtsExceptWM, 20)
wm2gray = makeAreaGrayer(WM2GRAY, wm_pts, 5)
wm2gray = makeAreaGrayer(wm2gray, WhitePtsExceptWM, 18)
WM2GRAD_GRAY = np.copy(wm2gray)
wm2gray = dither.floyd_steinberg(wm2gray / 255) * 255
wm2gray = cv.rotate(wm2gray, cv.ROTATE_90_CLOCKWISE)
epd.saveAsBlackWhiteImage(path_dst_B1, wm2gray)
print('Done')

100%|██████████| 1054124/1054124 [00:02<00:00, 514264.86it/s]


Done


In [31]:
# wm2gray2 = np.copy(WM2GRAY2)
# part1[100:300,100:300] = 255
# wm2gray2 = WM2GRAD_GRAY.copy()
# # wm2gray = makeAreaGrayer(WM2GRAY, wm_pts, 10)
# # wm2gray = makeAreaGrayer(wm2gray, WhitePtsExceptWM, 20)
# Ys = [1563,1653,1700]
# intvl = 6
# # for y in Ys:
# #     for i in [0,1,2,3]:
# #         wm2gray2[y - i * intvl:y, :] = wm2gray2[y - i * intvl:y, :] - i * 2
# #         wm2gray2[y + i * intvl:y + (i + 1) * intvl, :] = wm2gray2[y + i * intvl:y + (i + 1) * intvl, :] - i * 2
#     # WM2GRAY2[y-1*intvl:y+Margin,:] = WM2GRAY2[y-Margin:y+Margin,:]
# wm2gray2[1697:1700,:] = wm2gray2[1697:1700,:] - 2
# wm2gray2[1703:1706,:] = wm2gray2[1703:1706,:] - 2
# wm2gray2[1693:1696,:] = wm2gray2[1693:1696,:] - 3
# wm2gray2[1704:1707,:] = wm2gray2[1704:1707,:] - 3
# wm2gray2[1669:1692,:] = wm2gray2[1669:1692,:] - 6
# wm2gray2[1708:1756,:] = wm2gray2[1708:1756,:] - 6
# wm2gray2 = dither.floyd_steinberg(wm2gray2 / 255) * 255
# wm2gray2 = cv.rotate(wm2gray2, cv.ROTATE_90_CLOCKWISE)
# epd.saveAsBlackWhiteImage(path_dst_B1, wm2gray2)
# print('Done')

In [125]:
def makeAreaGrayer0000(img, ds_area_pts, ds_wm_pts, val):
    new_img =  np.copy(img)
    add_mat = np.zeros_like(img)
    add_mat = add_mat + val
    pts = []
    for p in tqdm.tqdm(ds_area_pts):
        if p in ds_wm_pts:
            continue
        x,y = p
        pts.append(p)
        # new_img[y*6:(y+1)*6-1,x*6:(x+1)*6-1] = new_img[y:y+6,x:x+6] + val
        # new_img[y*6:(y+1)*6-1,x*6:(x+1)*6-1] = 255#np.zeros_like(new_img[y:y+6,x:x+6]) + val
        # addAreaWithVal(new_img[y:y+6,x:x+6], 255)
        new_img[y*6:(y+1)*6-1,x*6:(x+1)*6-1] = cv.add(new_img[y*6:(y+1)*6-1,x*6:(x+1)*6-1], val)
    # vizPtsInArea(we_mat_downscale_after_we_darker.shape, [pts])
    return new_img

# New Deal

In [37]:
# Preparation
path_wkspace = 'Exp/star/'
path_src_home = path_wkspace + 'src/'
path_out_home = path_wkspace + 'out/'
# path_out_home = '\\\\192.168.1.9\\myshare\\IT8951\\pic\\exp\\star\\'
path_out_home = '\\\\192.168.137.114\\myshare\\IT8951\\pic\\exp\\opera\\'

src_type = '.jpg'
dst_type = '.bmp'
A = 'A'
B = 'B'
path_src_A = path_src_home + A + src_type
path_src_B = path_src_home + B + src_type
path_dst_A = path_out_home + A + dst_type
path_dst_B = path_out_home + B + dst_type

img_A = cv.imread(path_src_A, cv.IMREAD_GRAYSCALE)
img_B = cv.imread(path_src_B, cv.IMREAD_GRAYSCALE)

# img_A = epd.threshold(img_A, 238, 238, mode=3)
# img_B = epd.threshold(img_A, 238, 238, mode=3)

img_A_dither = dither.floyd_steinberg(img_A / 255) * 255
img_B_dither = dither.floyd_steinberg(img_B / 255) * 255
# img_A = cv.rotate(img_A, cv.ROTATE_90_CLOCKWISE)
# img_B = cv.rotate(img_B, cv.ROTATE_90_CLOCKWISE)
# # #
# epd.saveAsBlackWhiteImage(path_dst_A, img_A)
# epd.saveAsBlackWhiteImage(path_dst_B, img_B)

## White Edge

In [36]:
we_cnt_lut, we_pos_lut, we_mat012 = cwe.getWhiteEdgeCnt2(img_A_dither, img_B_dither)

In [38]:
black_mat2 = img_B.copy()
h,w = black_mat2.shape
black_pts = []
for y in range(h):
    for x in range(w):
        if black_mat2[y,x] > 30:
            black_mat2[y,x] = 0
        else:
            black_mat2[y,x] = 1
            black_pts.append((x,y))
white_mat2 = img_B.copy()
white_pts2 = []
h,w = white_mat2.shape
for y in range(h):
    for x in range(w):
        if white_mat2[y,x] > 245:
            white_mat2[y,x] = 1
            white_pts2.append((x,y))
        else:
            white_mat2[y,x] = 0
print(np.unique(white_pts2))
print(np.unique(black_mat2))

[   0    1    2 ... 1869 1870 1871]
[0 1]


In [39]:
def isInImage(x,y,h,w):
    return (x<w and x >= 0 and y < h and y >= 0)
def getDarkerMatAndPts8(wmat, wepts, threshold=4):
    dark_mat = np.zeros_like(wmat)
    dark_pts = []
    h,w = wmat.shape
    a = []
    for p in wepts:
        x,y = p
        adjacent_cnt = 0
        for i in [x,x-8,x+8]:
            for j in [y,y-8,y+8]:
                if i==x and j==y:
                    continue
                if isInImage(i,j,h,w) and wmat[j, i] == 1:
                    adjacent_cnt = adjacent_cnt + 1
        # a.append(adjacent_cnt)
        if adjacent_cnt < threshold:
            dark_mat[y,x] = 1
            dark_pts.append((x,y))
        # print(Counter(a))
    return dark_mat,dark_pts

In [40]:
darker_mat3, darker_pts2 = getDarkerMatAndPts8(white_mat2, list(we_cnt_lut.keys()),threshold=1)
# darker_mat3, darker_pts2 = getDarkerMatAndPts6(white_mat2, we_cnt_lut,threshold=1)
darker_mat3 = darker_mat3.astype('float64')

In [44]:
factor = 40#95
path_dst_BD2 = path_out_home + 'B1' + dst_type

whiteEdgeDarker2 = img_B.copy() - darker_mat3*factor
AfterWhiteEdgeDarker = whiteEdgeDarker2.copy()
whiteEdgeDarker2 = dither.floyd_steinberg(whiteEdgeDarker2/255)*255
whiteEdgeDarker2 = cv.rotate(whiteEdgeDarker2, cv.ROTATE_90_CLOCKWISE)
epd.saveAsBlackWhiteImage(path_dst_BD2, whiteEdgeDarker2)
print('Done')

Done


## White Minus

In [45]:
def getWhiteMinusCnt2(img_A, img_B):
    A = np.copy(img_A).astype('int16') / 255
    B = np.copy(img_B).astype('int16') / 255
    D = B - A
    D = D + 1
    D = D / 2
    h, w = A.shape
    wm_pts = []
    for y in range(h):
        for x in range(w):
            if D[y, x] == 1:
                wm_pts.append((x, y))
    return wm_pts, D.astype('int16')

In [46]:
wm_pts,wm_mat = getWhiteMinusCnt2(img_A_dither, img_B_dither)

In [47]:
white_mat = img_B.copy()
h,w = white_mat.shape
white_pts = []
for y in range(h):
    for x in range(w):
        if white_mat[y,x] > 238:
            white_mat[y,x] = 1
            white_pts.append((x,y))
        else:
            white_mat[y,x] = 0
print(np.unique(white_mat))

[0 1]


In [48]:
non_wm_white_pts = list(set(white_pts)-set(wm_pts))

In [49]:
vizPtsInArea(img_B.shape, [non_wm_white_pts])

In [50]:
def isInImage(x,y,h,w):
    return (x<w and x >= 0 and y < h and y >= 0)
def getDarkerMatAndPts(wmmat, nonwm_white_pts, threshold=4):
    dark_mat = np.zeros_like(wmmat)
    dark_pts = []
    h,w = wmmat.shape
    a = []
    for p in nonwm_white_pts:
        x,y = p
        adjacent_cnt = 0
        for i in [x,x-1,x+1]:
            for j in [y,y-1,y+1]:
                if i==x and j==y:
                    continue
                # print((i,j))
                if isInImage(i,j,h,w):
                    adjacent_cnt = adjacent_cnt + 1
        a.append(adjacent_cnt)
        if adjacent_cnt < threshold:
            dark_mat[y,x] = 1
            dark_pts.append((x,y))
        # print(Counter(a))
    return dark_mat,dark_pts

def getDarkerMatAndPts2(wmmat, nonwm_white_pts, threshold=4):
    dark_mat = np.zeros_like(wmmat)
    dark_pts = []
    h,w = wmmat.shape
    a = []
    for p in nonwm_white_pts:
        x,y = p
        adjacent_cnt = 0
        for i in [x,x-2,x+2]:
            for j in [y,y-2,y+2]:
                if i==x and j==y:
                    continue
                if isInImage(i,j,h,w) and wmmat[j,i] == 1:
                    adjacent_cnt = adjacent_cnt + 1
        # a.append(adjacent_cnt)
        if adjacent_cnt < threshold:
            dark_mat[y,x] = 1
            dark_pts.append((x,y))
        # print(Counter(a))
    return dark_mat,dark_pts

In [51]:
# dm, dp = getDarkerMatAndPts(wm_mat, non_wm_white_pts,threshold=9)
# darker_mat2, darker_pts = getDarkerMatAndPts2(wm_mat, non_wm_white_pts,threshold=2)
darker_mat_bd2, darker_pts_bd2 = getDarkerMatAndPts2(wm_mat, non_wm_white_pts,threshold=1)
# darker_mat2, darker_pts = getDarkerMatAndPts(wm_mat, non_wm_white_pts,threshold=8)


In [58]:
# path_dst_BD2 = path_out_home + 'B11' + dst_type
whiteMinusDarker = AfterWhiteEdgeDarker.copy()
factor = 8#20
# whiteMinusDarker = whiteMinusDarker + wm_mat*20
whiteMinusDarker = whiteMinusDarker - darker_mat_bd2*factor
# whiteMinusDarker = whiteMinusDarker + rest_mat*15
whiteMinusDarker = dither.floyd_steinberg(whiteMinusDarker/255)*255
whiteMinusDarker = cv.rotate(whiteMinusDarker, cv.ROTATE_90_CLOCKWISE)
epd.saveAsBlackWhiteImage(path_dst_BD2, whiteMinusDarker)
print('Done')

Done
